# Simulating observations

In [1]:
import os
import sys

sys.path.insert(0, os.path.join(".."))

from infobs import Simulator
from infobs.instruments import IdealInstrument, IRAM30mEMIR
from infobs.sampling import Constant, LogUniform

## Settings

In [2]:
lines = ["13c_o_j1__j0", "13c_o_j2__j1", "13c_o_j3__j2"]

n_samples = 5
linewidth = 10 # km/s
obstime = 1 # Minutes

## Samplers

In [3]:
samplers = {
    'Av': LogUniform(1, 25),
    'G0': LogUniform(1e1, 1e3),
    'Pth': LogUniform(1e5, 5e6),
    'angle': Constant(0.)
}

**Warning:** Sampler intervals should be included in the emulator's range of validity, i.e.,
- $A_v^{\text{max}} \in \left[1,\,40\right]$,
- $G_0 \in \left[1,\,10^5\right]$,
- $P_{\text{th}} \in \left[10^5,\,10^9\right]$,
- $\alpha \in \left[0,\,60\right]$.

The chosen sampler corresponds roughly to the range of values within the horsehead pillar. Note that here a loguniform distribution is used while in practice the distributions are more sophisticated (see `sampling.ipynb` example notebook).

What's more, here the sampler chosen for the angle $\alpha$ is constant at 0, meaning that the PDR is assumed to be always observed face-on.

## Ideal instrument

In [5]:
ideal = IdealInstrument()
simulator = Simulator(ideal)

simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,20.400084,133.680075,1.015463e+06,0.0,1.0,36.937608,39.969414,27.257519
1,11.378295,25.935572,2.670705e+06,0.0,1.0,26.884806,31.325124,20.191394
2,4.480663,12.497250,1.642648e+06,0.0,1.0,11.387329,17.804883,11.414140
3,15.728752,489.278136,2.598168e+06,0.0,1.0,32.345114,43.439317,33.491593
4,7.838844,591.907347,1.444927e+06,0.0,1.0,16.779969,28.537100,21.911306


## IRAM 30m EMIR

In [7]:
emir = IRAM30mEMIR(linewidth)
simulator = Simulator(emir)

simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,21.105386,214.765108,2.679169e+06,0.0,1.0,38.610096,53.651712,49.828553
1,3.822963,32.253592,1.603479e+05,0.0,1.0,12.212951,8.611446,2.597003
2,9.540999,24.221859,6.181714e+05,0.0,1.0,20.466533,25.108435,22.254860
3,14.160185,903.575858,7.250881e+05,0.0,1.0,31.336987,40.857115,32.116903
4,2.710980,109.378006,4.150705e+05,0.0,1.0,0.737521,3.286243,-14.759389


## Changing integration time

In [8]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    10 * obstime
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,1.276200,238.242588,4.636076e+05,0.0,1.0,-0.564608,0.482551,-1.291372
1,14.426569,31.322129,2.902063e+05,0.0,1.0,30.065746,29.615785,7.698290
2,4.029577,275.026140,3.246161e+06,0.0,1.0,6.467323,15.924636,21.363641
3,4.487343,39.257576,2.563935e+05,0.0,1.0,8.931628,15.054640,5.259377
4,5.163839,18.706052,1.745420e+06,0.0,1.0,13.797872,16.383609,14.685765


## Adding a scaling parameter $\kappa$

The $\kappa$ scaling parameter is used to integrated beam dilution ($\kappa < 1$) and geometry uncertainty ($\kappa > 1$). It is always positive.

In [9]:
kappa_sampler = LogUniform(0.1**0.5, 10**0.5)

simulator.simulate(
    n_samples,
    samplers,
    lines,
    10 * obstime,
    kappa=kappa_sampler
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,1.258990,18.522247,9.060345e+05,0.0,1.495605,1.622135,2.797728,-0.356867
1,9.912180,53.154489,3.736430e+06,0.0,0.413825,8.163709,14.444331,-1.590261
2,21.410173,13.718167,1.364351e+05,0.0,0.416120,15.201492,13.618959,8.545133
3,23.273107,137.002359,3.416175e+06,0.0,1.528925,66.539683,70.725880,45.222883
4,1.792967,21.910649,1.997814e+06,0.0,1.056051,2.115748,5.640110,2.994744


## Seeding

In [10]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime,
    kappa=kappa_sampler
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,9.201223,88.138338,1.127664e+06,0.0,0.524591,10.408096,15.455927,6.343986
1,7.213161,70.253347,1.246069e+05,0.0,1.146578,22.330642,22.138828,-1.424833
2,5.658353,13.299521,5.791259e+05,0.0,0.519762,5.669018,10.070234,23.680238
3,2.988243,22.137981,3.286108e+06,0.0,0.632196,1.848720,11.568506,-17.065037
4,3.452220,10.287679,2.064384e+05,0.0,0.730925,7.518060,6.060717,-7.892146


In [11]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime,
    kappa=kappa_sampler,
    seed=0
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,5.850707,195.788972,2.213691e+06,0.0,0.386482,7.773508,7.470687,11.525396
1,9.995208,75.019544,7.917291e+05,0.0,0.331298,5.980112,3.396069,19.040374
2,6.960285,607.499607,9.227641e+05,0.0,2.150898,33.660216,47.636578,78.493467
3,5.777155,845.912653,3.737326e+06,0.0,1.897391,15.426901,35.160349,27.747462
4,3.910605,58.463261,1.320347e+05,0.0,2.344294,21.657290,25.216067,11.163777


In [12]:
simulator.simulate(
    n_samples,
    samplers,
    lines,
    obstime,
    kappa=kappa_sampler,
    seed=0
)

,Av,G0,Pth,angle,kappa,13c_o_j1__j0,13c_o_j2__j1,13c_o_j3__j2
0,5.850707,195.788972,2.213691e+06,0.0,0.386482,7.773508,7.470687,11.525396
1,9.995208,75.019544,7.917291e+05,0.0,0.331298,5.980112,3.396069,19.040374
2,6.960285,607.499607,9.227641e+05,0.0,2.150898,33.660216,47.636578,78.493467
3,5.777155,845.912653,3.737326e+06,0.0,1.897391,15.426901,35.160349,27.747462
4,3.910605,58.463261,1.320347e+05,0.0,2.344294,21.657290,25.216067,11.163777
